In [1]:
import os
import json
import numpy as np
import datetime
import pandas as pd
import pymysql
import requests
import requests.exceptions
import mysql.connector
from urllib.request import urlopen
from ast import literal_eval
import io
import tabulate

import pymongo
from sqlalchemy import create_engine

In [2]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "poggers"
db_name = 'ds_capstone'

host_name = "localhost"
ports = {"mongo" : 27017, "mysql" : 3306}

src_dbname = "ds_capstone"
dst_dbname = "ds_capstone"

url = 'https://query.data.world/s/3yhssk5zjeadq2qdammomhyo3vca33' ## Data Source
date_path = r'C:\Users\xXDRP\DS_3002\DS_3002_Working_Directory\DS_3002_Origin\Date.csv'
### Manually import date table created in MySQL Workbench since I cannot get communication between MySQL and MongoDB ###

In [4]:
def json_df(df):
    df = df.reset_index()
    s = (df.applymap(type) == list).all()
    list_columns = s[s].index.tolist()
    
    s = (df.applymap(type) == dict).all()
    dict_columns = s[s].index.tolist()

    
    while len(list_columns) > 0 or len(dict_columns) > 0:
        new_columns = []

        for col in dict_columns:
            horizontal = pd.json_normalize(df[col]).add_prefix(f'{col}.')
            horizontal.index = df.index
            df = pd.concat([df, horizontal], axis=1).drop(columns=[col])
            new_columns.extend(horizontal.columns) # inplace

        for col in list_columns:
            df = df.drop(columns=[col]).join(df[col].explode().to_frame())
            new_columns.append(col)

        s = (df[new_columns].applymap(type) == list).all()
        list_columns = s[s].index.tolist()

        s = (df[new_columns].applymap(type) == dict).all()
        dict_columns = s[s].index.tolist()
    return df

In [5]:
response = requests.get(url)
rep = response.json()['data'] ## Get 'Data' info from source

In [7]:
df_old = pd.DataFrame(rep)
df = json_df(df_old)
pd.set_option('display.max_columns', None)
df.drop_duplicates(subset=['id'],inplace=True)
df
### Getting continous, streaming cryptocurrency data ###

,index,id,name,symbol,slug,num_market_pairs,date_added,max_supply,circulating_supply,total_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,last_updated,quote.USD.price,quote.USD.volume_24h,quote.USD.volume_change_24h,quote.USD.percent_change_1h,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.last_updated,tags
0,0,1,Bitcoin,BTC,bitcoin,9424,2013-04-28T00:00:00.000Z,2.100000e+07,1.903797e+07,1.903797e+07,None,1,NaN,NaN,2022-05-12T14:27:00.000Z,28615.961090,8.073486e+10,12.6431,0.495325,-8.235843,-25.992064,-28.532288,-26.402719,-34.435133,5.447898e+11,44.8117,6.009352e+11,2022-05-12T14:27:00.000Z,mineable
1,1,1027,Ethereum,ETH,ethereum,5708,2015-08-07T00:00:00.000Z,NaN,1.207489e+08,1.207489e+08,None,2,NaN,NaN,2022-05-12T14:26:00.000Z,1949.085022,4.940417e+10,13.4138,0.008731,-16.667697,-32.033220,-35.692306,-24.056075,-37.479349,2.353499e+11,19.4910,2.353499e+11,2022-05-12T14:26:00.000Z,mineable
2,2,825,Tether,USDT,tether,33294,2015-02-25T00:00:00.000Z,NaN,8.279679e+10,8.671362e+10,"{'id': 1, 'name': 'Ethereum', 'symbol': 'USDT'...",3,NaN,NaN,2022-05-12T14:26:00.000Z,0.994829,1.716360e+11,7.9625,0.042185,-0.353646,-0.526781,-0.543350,-0.534828,-0.571476,8.236864e+10,6.7980,8.626522e+10,2022-05-12T14:26:00.000Z,payments
3,3,3408,USD Coin,USDC,usd-coin,3961,2018-10-08T00:00:00.000Z,NaN,4.912217e+10,4.912217e+10,"{'id': 1, 'name': 'Ethereum', 'symbol': 'USDC'...",4,NaN,NaN,2022-05-12T14:26:00.000Z,1.001818,2.155018e+10,88.0825,-0.036683,0.119429,0.210507,0.246931,0.191695,0.294605,4.921149e+10,4.0701,4.921149e+10,2022-05-12T14:26:00.000Z,medium-of-exchange
4,4,1839,BNB,BNB,bnb,839,2017-07-25T00:00:00.000Z,1.651168e+08,1.632770e+08,1.632770e+08,None,5,NaN,NaN,2022-05-12T14:26:00.000Z,271.710437,5.140151e+09,40.1139,-0.751842,-9.259466,-31.231766,-33.892023,-27.010958,-34.723884,4.436406e+10,3.6614,4.486395e+10,2022-05-12T14:26:00.000Z,marketplace
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,873,NEM,XEM,nem,122,2015-04-01T00:00:00.000Z,9.000000e+09,9.000000e+09,9.000000e+09,None,96,NaN,NaN,2022-05-12T14:26:00.000Z,0.041009,4.339947e+07,11.2995,-1.069592,-24.431690,-53.814457,-58.273399,-56.112333,-64.770156,3.690769e+08,0.0306,3.690769e+08,2022-05-12T14:26:00.000Z,poi
96,96,8677,Symbol,XYM,symbol,38,2021-03-03T00:00:00.000Z,9.000000e+09,5.582460e+09,7.967989e+09,None,97,NaN,NaN,2022-05-12T14:26:00.000Z,0.063394,1.733951e+07,9.1136,-0.024457,-15.656473,-32.532505,-44.221434,-52.504342,-65.135623,3.538967e+08,0.0293,5.705495e+08,2022-05-12T14:26:00.000Z,NaN
97,97,1727,Bancor,BNT,bancor,310,2017-06-18T00:00:00.000Z,NaN,2.555453e+08,2.555453e+08,"{'id': 1, 'name': 'Ethereum', 'symbol': 'BNT',...",98,NaN,NaN,2022-05-12T14:26:00.000Z,1.340970,4.716505e+07,22.1872,0.453574,-15.279395,-34.984069,-41.616969,-37.638639,-50.516743,3.426784e+08,0.0284,3.426784e+08,2022-05-12T14:26:00.000Z,marketplace
98,98,1808,OMG Network,OMG,omg,284,2017-07-14T00:00:00.000Z,1.402454e+08,1.402454e+08,1.402454e+08,"{'id': 1, 'name': 'Ethereum', 'symbol': 'OMG',...",99,NaN,NaN,2022-05-12T14:26:00.000Z,2.419425,2.889091e+08,4.6302,-2.045940,-18.959891,-39.669826,-45.511233,-39.576328,-53.067497,3.393132e+08,0.0281,3.393132e+08,2022-05-12T14:26:00.000Z,medium-of-exchange


In [8]:
df_changes = df[['id','name','quote.USD.percent_change_1h','quote.USD.percent_change_24h','quote.USD.percent_change_7d','quote.USD.percent_change_30d']]
df_changes.rename(columns = {'quote.USD.percent_change_1h':'Percent_Change_1h','quote.USD.percent_change_24h':'Percent_Change_24h','quote.USD.percent_change_7d':'Percent_Change_7d','quote.USD.percent_change_30d':'Percent_Change_30d'})

,id,name,Percent_Change_1h,Percent_Change_24h,Percent_Change_7d,Percent_Change_30d
0,1,Bitcoin,0.495325,-8.235843,-25.992064,-28.532288
1,1027,Ethereum,0.008731,-16.667697,-32.033220,-35.692306
2,825,Tether,0.042185,-0.353646,-0.526781,-0.543350
3,3408,USD Coin,-0.036683,0.119429,0.210507,0.246931
4,1839,BNB,-0.751842,-9.259466,-31.231766,-33.892023
...,...,...,...,...,...,...
95,873,NEM,-1.069592,-24.431690,-53.814457,-58.273399
96,8677,Symbol,-0.024457,-15.656473,-32.532505,-44.221434
97,1727,Bancor,0.453574,-15.279395,-34.984069,-41.616969
98,1808,OMG Network,-2.045940,-18.959891,-39.669826,-45.511233


In [9]:
df_sales = df[['id','name','circulating_supply','quote.USD.price']]
df_sales.rename(columns = {'quote.USD.price':'Price_USD'},inplace=True)
df_sales

C:\Users\xXDRP\AppData\Local\Temp/ipykernel_13956/3929784604.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales.rename(columns = {'quote.USD.price':'Price_USD'},inplace=True)


,id,name,circulating_supply,Price_USD
0,1,Bitcoin,1.903797e+07,28615.961090
1,1027,Ethereum,1.207489e+08,1949.085022
2,825,Tether,8.279679e+10,0.994829
3,3408,USD Coin,4.912217e+10,1.001818
4,1839,BNB,1.632770e+08,271.710437
...,...,...,...,...
95,873,NEM,9.000000e+09,0.041009
96,8677,Symbol,5.582460e+09,0.063394
97,1727,Bancor,2.555453e+08,1.340970
98,1808,OMG Network,1.402454e+08,2.419425


In [10]:
df_coins = df[['id','name','symbol','date_added','quote.USD.last_updated']].copy()
df_coins['quote.USD.last_updated'] = pd.to_datetime(df_coins['quote.USD.last_updated'])
df_coins['date_added'] = pd.to_datetime(df_coins['date_added'])
df_coins.rename(columns = {'quote.USD.last_updated':'Last_Updated'},inplace=True)
df_coins

,id,name,symbol,date_added,Last_Updated
0,1,Bitcoin,BTC,2013-04-28 00:00:00+00:00,2022-05-12 14:27:00+00:00
1,1027,Ethereum,ETH,2015-08-07 00:00:00+00:00,2022-05-12 14:26:00+00:00
2,825,Tether,USDT,2015-02-25 00:00:00+00:00,2022-05-12 14:26:00+00:00
3,3408,USD Coin,USDC,2018-10-08 00:00:00+00:00,2022-05-12 14:26:00+00:00
4,1839,BNB,BNB,2017-07-25 00:00:00+00:00,2022-05-12 14:26:00+00:00
...,...,...,...,...,...
95,873,NEM,XEM,2015-04-01 00:00:00+00:00,2022-05-12 14:26:00+00:00
96,8677,Symbol,XYM,2021-03-03 00:00:00+00:00,2022-05-12 14:26:00+00:00
97,1727,Bancor,BNT,2017-06-18 00:00:00+00:00,2022-05-12 14:26:00+00:00
98,1808,OMG Network,OMG,2017-07-14 00:00:00+00:00,2022-05-12 14:26:00+00:00


In [11]:
def get_sql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    conn = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, conn);
    conn.close()
    
    return dframe


def get_mongo_dataframe(user_id, pwd, host_name, port, db_name, collection, query):
    '''Create a connection to MongoDB, with or without authentication credentials'''
    if user_id and pwd:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db_name)
        client = pymongo.MongoClient(mongo_uri)
    else:
        conn_str = f"mongodb://{host_name}:{port}/"
        client = pymongo.MongoClient(conn_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()
    
def mongoimport(csv_path, db_name, coll_name, db_url='localhost', db_port=port):
    """ Imports a csv file at path csv_name to a mongo colection
    returns: count of the documents in the new collection
    """
    client = pymongo.MongoClient(db_url, db_port)
    db = client[db_name]
    coll = db[coll_name]
    data = pd.read_csv(csv_path)
    payload = json.loads(data.to_json(orient='records'))
    coll.drop()
    coll.insert_many(payload)
    
def mongoimport_df(df, db_name, coll_name, db_url='localhost', db_port=port):
    """ Imports adf at path df to a mongo colection"""
    client = pymongo.MongoClient(db_url, db_port)
    db = client[db_name]
    coll = db[coll_name]
    data = df
    payload = json.loads(data.to_json(orient='records'))
    coll.drop()
    coll.insert_many(payload)

In [12]:
### Import Dataframes into MongoDB ###
mongoimport_df(df_coins,dst_dbname,'Currencies',host_name,27017)

In [13]:
mongoimport_df(df_changes,dst_dbname,'Market_Flux',host_name,27017)

In [14]:
mongoimport_df(df_sales,dst_dbname,'Crypto_Sales',host_name,27017)

In [15]:
mongoimport(date_path,dst_dbname,'Date',host_name,27017)

In [16]:
query = {'name':{'$in':['Bitcoin','Ethereum','Tether','USD Coin','BNB','GateToken','Dogecoin']}} # Select all elements (columns), and all documents (rows).

port = ports["mongo"]
collection = "Crypto_Sales"
### Get Dataframe from MongoDB to create Fact Table ###
df_fact_sales = get_mongo_dataframe(None, None, host_name, port, src_dbname, collection, query)
df_fact_sales.rename(columns = {'quote.USD.price':"Price_USD"},inplace=True)
df_fact_sales

,id,name,circulating_supply,Price_USD
0,1,Bitcoin,1.903797e+07,28615.961090
1,1027,Ethereum,1.207489e+08,1949.085022
2,825,Tether,8.279679e+10,0.994829
3,3408,USD Coin,4.912217e+10,1.001818
4,1839,BNB,1.632770e+08,271.710437
5,74,Dogecoin,1.326708e+11,0.077633
6,4269,GateToken,7.833048e+07,4.881578


In [17]:
### Import New Fact Table to MongoDB ###
mongoimport_df(df_fact_sales,dst_dbname,'Fact_Crypto_Sales',host_name,27017)

In [18]:
### Manually Create CSV file to import into Power BI since I cannot connect from either MongoDB or MySQL Workbench ###
df_fact_sales.to_csv("Fact_Sales.csv")